Focal Loss
===

# 1.单阶段目标检测存在的问题

虽然单阶段目标检测的速度很快，但是其精度往往比较低，究其原因，在于两个方面

## 1.1.正样本(Postive Example)和负样本(Negative Example)的不平衡

Negative example的数量过多，导致Postive example的loss被覆盖，就算Postive example的loss非常大也会被数量庞大的negative example中和掉，这这些positive example往往是我们要检测的前景区域

## 1.2.难样本(Hard Example)和易样本(Easy Example)的不平衡

Hard example往往是前景和背景区域的过渡部分，因为这些样本很难区分，所以叫做Hard Example。剩下的那些Easy example往往很好计算，导致模型非常容易就收敛了。但是损失函数收敛了并不代表模型效果好，因为我们其实更需要把那些hard example训练好。

![Images](Images/03/02/03_04_001.jpg)

Faster R-CNN之所以能解决两个不平衡问题是因为其采用了下面两个策略：

1. 根据IoU采样候选区域，并将正负样本的比例设置成1:1。这样就解决了正负样本不平衡的问题；
2. 根据score过滤掉easy example，避免了训练loss被easy example所支配的问题。

# 2.Focal Loss

大神采用的解决方案是基于交叉熵提出了一个新的损失函数Focal Loss(FL)

$$FL(p_t)=-\alpha_t(1-p_t)^\gamma log(p_t)$$

FL是一个尺度动态可调的交叉熵损失函数，在FL中有两个参数$\alpha_t$和$\gamma$，其中$\alpha_t$的主要作用是解决正负样本的不平衡问题，$\gamma$主要是解决难易样本不平衡的问题。Focal Loss是交叉熵损失的改进版本，一个二分类交叉熵可以表示为：

$$CE(p,y)=\begin{cases}
-log(p) & y=1 \\
-log(1-p) & otherwise
\end{cases}$$

上面公式可以简写成

$$CE(p,y)=CE(p_t)=-log(p_t)$$

其中

$$p_t=\begin{cases}
p & y=1
1-p & otherwise
\end{cases}$$

## 2.1.$\alpha$解决正负样本不平衡

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
def one_hot(index, classes):
    size = index.size() + (classes,)
    view = index.size() + (1,)
    mask = torch.Tensor(*size).fill_(0)
    index = index.view(*view)
    ones = 1.
    if isinstance(index, Variable):
        ones = Variable(torch.Tensor(index.size()).fill_(1))
        mask = Variable(mask, volatile=index.volatile)
    return mask.scatter_(1, index, ones)
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
    def forward(self, input, target):
        y = one_hot(target, input.size(-1))
        logit = F.softmax(input)
        logit = logit.clamp(self.eps, 1. - self.eps)
        loss = -1 * y * torch.log(logit) # cross entropy
        loss = loss * (1 - logit) ** self.gamma # focal loss
        return loss.sum()

## 4.3.RetinaNet
利用Focal Loss，基于ResNet和Feature Pyramid Net(FPN)设计了一种新的one-stage检测框架，命名为RetinaNet

